<h4>Creator: Gusthema</h4>

<h5>Imports dependencies</h5>

In [1]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

C:\Users\hyanl\AppData\Local\Temp\ipykernel_4220\2019373907.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'tensorflow'

<h5>Load dataset</h5>

In [ ]:
train_df = pd.read.csv("dataset/titanic/extracted/train.csv")
serving_df = pd.read.csv("dataset/titanic/extracted/test.csv")

train_df.head(10)

<h5>Prepare dataset</h5>

In [ ]:
def preprocess(df):
    df = df.copy()

    def normalize_name(x):
        return " ".join([v.strip
                         (",()[].\"'") for v in x.split("")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
    
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)

    df["Ticket_number"] = df["Ticket"].apply(ticket_number)

    df["Ticket_item"] = df["Ticket"].apply(ticket_item)

    return df

preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

In [ ]:
input_features = list(preprocessed_train_df.columns)

input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")

print(f"Input features: {input_features}")

<h5>Convert Pandas dataset to TensorFlow Dataset</h5>

In [ ]:
def tokenize_names(features, labels=None):

    features["Name"] = tf.strings.plit(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)    

<h5>Train model with default parameters</h5>